In [17]:
import sys

sys.path.append("/u/li19/diffusers_with_dataloader/examples/audio_generation/audio_gen_files")
from AudiosetDataset import AudiosetDataset
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import StableDiffusionPipeline
import torch

device = "cuda"

In [18]:
model_path = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_path, device_map=None, safety_checker=None)
pipe.to("cuda")



/u/li19/data_folder/anaconda3/envs/diffuse/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.15.1",
  "feature_extractor": [
    "transformers",
    "CLIPFeatureExtractor"
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [19]:
dataset_json_file = "/u/li19/data_folder/AudioTaggingDoneRight/egs/audioset/data/datafiles/delta_bal_alpaca_train.json"
latent_folder = "/u/li19/data_folder/AudioTaggingDoneRight/egs/audioset/data/datafiles/bal_latent_raw"
wav_folder = "/u/li19/data_folder/audioSetAudio/balance_wav"
tokenizer = CLIPTokenizer.from_pretrained(
"CompVis/stable-diffusion-v1-4", subfolder="tokenizer"
    )

device = "cpu"
dataset = AudiosetDataset(
    dataset_json_file, 
    latent_folder=latent_folder, 
    tokenizer=tokenizer, 
    device=device, 
    wav_folder=wav_folder,
    dtype=torch.float32,
    channels=1
)

dataloader = torch.utils.data.DataLoader(dataset)

In [20]:
sample = dataset[0]
latent = sample["latent"]
reshaped = latent.reshape((latent.shape[0], 8, 21, 24))
print(latent.shape)
print(reshaped.shape)

torch.Size([1, 8, 504])
torch.Size([1, 8, 21, 24])


In [24]:
noise = torch.randn_like(reshaped)
t_steps= torch.randn([8])
fake_embeds = torch.randn([8,77,768])
print(noise.shape)
print(t_steps.shape)
print(fake_embeds.shape)

torch.Size([1, 8, 21, 24])
torch.Size([8])
torch.Size([8, 77, 768])


In [27]:
#Testing if our reshape works in the unet

pipe.unet(noise[0], t_steps, fake_embeds)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)